## Propósito:

Esse notebook traz um estudo sobre o desenvolvimento de um algoritimo que visa prever o resultado de embates entre dois times de futebol a partir de sua escalação e jogadores.

Durante a leitura você percebera que dezenas de testes e tratamentos de dados foram realizados, visando trazer o máximo de acurácia e precisão possível.

In [1]:
import pandas as pd
from sklearn.cluster import KMeans
import plotly.express as px
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import sklearn.linear_model as lm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from sklearn.ensemble import RandomForestClassifier

In [2]:
def normalize_data(data):
  min_data = np.min(data)
  max_data = np.max(data)

  return (data - min_data) / (max_data - min_data)

def dect_Outliers(dados):
  q1 = np.percentile(dados,25)
  q3 = np.percentile(dados,75)

  iiq = q3 - q1   

  min = q1 - 2 * iiq
  max = q3 + 2 * iiq

  outliers = [x for x in dados if x < min or x > max]

  return outliers

def trat_outlier_per_quartis(coluna, fator):
    # Calcula os quartis
    Q1 = coluna.quantile(0.25)
    Q3 = coluna.quantile(0.75)
    IQR = Q3 - Q1  # Intervalo interquartil

    # Calcula os limites de outliers
    limite_inferior = Q1 - fator * IQR
    limite_superior = Q3 + fator * IQR

    # Trata os outliers
    coluna_tratada = coluna.apply(lambda x: Q1 if x < limite_inferior else (Q3 if x > limite_superior else x))

    return coluna_tratada

In [3]:
partidas_serie_a = pd.read_json('../../assets/brasileirao_serie_a_2024.json')
players_result_per_escalation = pd.read_csv('./player_results_per_escalation.csv')

matches_ibm = pd.read_csv('../../assets/matches.csv')
matches_fifa = pd.json_normalize(partidas_serie_a['Results'])

matches_fifa.to_csv('./matches_fifa.csv', index=False)

players = pd.read_csv('../../assets/players_atualizada.csv')

events = pd.read_json('../../assets/matches_events.json')
events = events[[
  'IdMatch',
  'Event'
]]
events = events.explode('Event')

players = players.apply(lambda col: col.str.replace(',', '.') if col.dtype == 'object' else col)

# substitui NaN por 0
players = players.fillna(0)

# remove a coluna eventos que esteja NaN
events = events.dropna(subset=['Event'])

## Tratamento da tabela players

Visando remover ruídos no modelo, estamos utilizando um sistema de remoção de outliers e normalização de colunas.

> Os outliers são dados que se diferenciam drasticamente de todos os outros. Em outras palavras, um outlier é um valor que foge da normalidade e que pode (e provavelmente irá) causar anomalias nos resultados obtidos por meio de algoritmos e sistemas de análise.

ANALYTICS, A. A. Outliers, o que são e como tratá-los em uma análise de dados? Disponível em: <https://aquare.la/o-que-sao-outliers-e-como-trata-los-em-uma-analise-de-dados/>.

<br />

> A normalização é focada na prevenção de problemas com repetição e atualização de dados, assim como o cuidado com a integridade dos dados. Este conceito foi apresentado originalmente em um artigo científico publicado pela IBM de autoria do matemático Edgar F. Codd, intitulado "Um modelo de dados relacionais para grandes bancos de dados compartilhados" (1970). Codd se centra nos valores dos elementos relacionados no banco de dados, não em ligações ou agrupamentos específicos.
 
Normalização em Banco de Dados - Estrutura. Disponível em: <https://www.alura.com.br/artigos/normalizacao-banco-de-dados-estrutura>.

<br />

Realizamos a seleção das colunas de jogadores visando destinguir diferentes grupos para entender como funciona a dinamica de jogo entre os times.
Analisamos os dados e identificamos as seguinte colunas como primordiais:

In [4]:
sample_players = players[[
  'goals_overall',
  'crosses_per_game_overall',
  'dribbles_per_game_overall',
  'interceptions_per_game_overall',
  'assists_per_game_overall',
  'penalty_goals',
]].apply(pd.to_numeric, errors='coerce')

total_features = len(players.columns)
outliers_list = []

for i, col in enumerate(sample_players.columns):
  outliers = dect_Outliers(sample_players[col])
  if outliers:
    outliers_list.append({
      'columnName': col,
      'outliers': outliers
    })

## Clusterização de dados
Usando as colunas encontradas anteriormente, iniciamos um processo que chamamos de clusterização. A cluterização tem como objetivo identificar "Clusters" de jogadores e entender como eles se relacionam entre si.

Para realizarmos a clusterização, utilizamos KMeans, que é um modelo não supervisionado que encontra correlações entre as colunas e os dados, atribuindo uma classe aos mesmos.

> É um algoritmo de clusterização (agrupamento) não supervisionado, baseado na definição de centroides que representam clusters. O “K” refere-se ao número de centroides (clusters) definidos previamente e o “Means” à média dos pontos em cada cluster que determina a posição de seu centroide.

ESCOLA DNC. K-Means: conheça esse algoritmo poderoso para clusterização - Blog DNC. Disponível em: <https://www.escoladnc.com.br/blog/kmeans-o-algoritmo-poderoso-para-clusterizacao-nao-supervisionada/#:~:text=O%20K%2DMeans%20%C3%A9%20um>. Acesso em: 13 set. 2024.

In [5]:
model = KMeans(n_clusters=3)
kmeans_features = sample_players.iloc[:, :4]
model.fit(kmeans_features)  
sample_players['cluster'] = model.predict(kmeans_features)
sample_players['cluster'].value_counts()
sample_players['cluster'] = sample_players['cluster'].astype(str)

In [ ]:
fig = px.scatter_matrix(sample_players, dimensions=sample_players.iloc[:, :7].columns, color='cluster', height=2000)
fig.show()

As relações acima nos ajuda a encontrar as seguintes hipóteses:

1 - Há uma relação clara entre jogadores que cruzam e driblam mais (dribbles_per_game_overall x crosses_per_game_overall)
<br>
2 - Há uma relação clara entre jogadores que cruzam e interceptam (crosses_per_game_overall x interceptions_per_game_overall)

In [ ]:
graphs = px.scatter_3d(
  x=sample_players['goals_overall'], 
  y=sample_players['interceptions_per_game_overall'], 
  z=sample_players['crosses_per_game_overall'], 
  color=sample_players['cluster'])

# change x y z labels
graphs.update_layout(scene = dict(
  xaxis_title='Gols',
  yaxis_title='Interceptações',
  zaxis_title='Dribles'
))

1 - Jogadores que driblam menos e fazem menos interceptações, estão mais propicios a fazer gols

In [ ]:
graph = px.scatter_3d(
  x=sample_players['goals_overall'], 
  y=sample_players['crosses_per_game_overall'], 
  z=sample_players['dribbles_per_game_overall'], 
  color=sample_players['cluster'])

# change x y z labels
graph.update_layout(scene = dict(
  xaxis_title='Gols',
  yaxis_title='Cruzamentos',
  zaxis_title='Dribles'
))


In [ ]:
graph = px.scatter_3d(
  x=sample_players['goals_overall'], 
  y=sample_players['crosses_per_game_overall'], 
  z=sample_players['interceptions_per_game_overall'], 
  color=sample_players['cluster'])

# change x y z labels
graph.update_layout(scene = dict(
  xaxis_title='Gols',
  yaxis_title='Cruzamentos',
  zaxis_title='Interceptações'
))


In [ ]:
graph = px.scatter_3d(
  x=sample_players['goals_overall'], 
  y=sample_players['crosses_per_game_overall'], 
  z=sample_players['interceptions_per_game_overall'], 
  color=sample_players['cluster'])
# change x y z labels
graph.update_layout(scene = dict(
    xaxis_title='Gols',
    yaxis_title='Cruzamentos',
    zaxis_title='Interceptações'
))


## Redução de dimencionalidade

Após encontrar as correlações entre os jogadores e seus diferentes clusters, buscamos reduzir o número de variáveis também conhecidas como features. Para isso utilizamos o PCA (Principal Component Analysis)

> Análise de Componentes Principais (PCA) é uma técnica fundamental em análise de dados e aprendizado de máquina que tem aplicações em uma ampla gama de campos, incluindo ciência de dados, visão computacional, reconhecimento de padrões e muito mais. PCA é utilizada para reduzir a dimensionalidade dos dados, preservando ao máximo a variabilidade presente nos mesmos.

HUGO, V. Análise de Componentes Principais (PCA) - Victor Hugo F. Francheto - Medium. Disponível em: <https://medium.com/@victor.h.f.francheto/an%C3%A1lise-de-componentes-principais-pca-a1e7361de059>. Acesso em: 13 set. 2024.

In [ ]:
pca = PCA(n_components=2)
features = sample_players.drop(columns=['cluster'], axis=1)
pca.fit(features.iloc[:, :7])
pca_samples = pca.transform(features.iloc[:, :7])
pca_samples = pd.DataFrame(
  pca_samples, 
  columns=[
    'player_PC1', 
    'player_PC2'
])
pca_samples['cluster'] = sample_players['cluster']
pca_samples['current_club'] = players['current_club']
pca_samples['position'] = players['position']
pca_samples['full_name'] = players['full_name']


teams_players_cluster = pca_samples.groupby('current_club')['cluster'].value_counts().unstack(fill_value=0) 

# Contagem de clubes
club_counts = pca_samples['current_club'].value_counts()
teams_players_cluster = teams_players_cluster.div(club_counts, axis=0)

# Renomear as colunas para refletir os clusters
teams_players_cluster.columns = [f'cluster_{col}' for col in teams_players_cluster.columns]
teams_players_cluster = teams_players_cluster.reset_index()

# Plota gráfico Scree
fig = px.bar(x=range(1, len(pca.explained_variance_ratio_)+1),
             y=pca.explained_variance_ratio_,
             title='Gráfico Scree (Relação de Percentual de Variância e Componente Principal)',
             labels={'x':'Componente Principal', 'y':'Percentual de variância'})
fig.show()

In [ ]:
px.scatter(
  pca_samples, 
  x='player_PC1', 
  y='player_PC2',
  color='cluster'
)

In [ ]:
teams = pd.read_csv('../../assets/teams_atualizada.csv')
teams = teams[[
  'shots_on_target', 
  'shots',
  'average_possession', 
  'average_total_goals_per_match', 
  'win_percentage', 
  'fts_percentage', 
  'common_name',
  'points_per_game',
  'fouls',
  'wins',
  'draws',
  'losses'
]]

teams['points_per_game'] = trat_outlier_per_quartis(teams['points_per_game'], 2)
teams

In [ ]:
teams_pca = PCA(n_components=2)
teams_features = teams.drop(columns=['common_name'], axis=1)
teams_pca.fit(teams_features)
pca_teams_samples = teams_pca.transform(teams_features)
pca_teams_samples = pd.DataFrame(pca_teams_samples, columns=['team_PC1', 'team_PC2'])
pca_teams_samples['common_name'] = teams['common_name']

pca_teams_samples = pca_teams_samples.merge(teams_players_cluster, left_on='common_name', right_on='current_club')

# Plota gráfico Scree
fig = px.bar(x=range(1, len(teams_pca.explained_variance_ratio_)+1),
             y=teams_pca.explained_variance_ratio_,
             title='Gráfico Scree (Relação de Percentual de Variância e Componente Principal)',
             labels={'x':'Componente Principal', 'y':'Percentual de variância'})
fig.show()

In [ ]:
model = KMeans(n_clusters=3)
kmeans_teams_features = pca_teams_samples[['team_PC1', 'team_PC2']]
model.fit(kmeans_teams_features)  
pca_teams_samples['team_cluster'] = model.predict(kmeans_teams_features)
pca_teams_samples['team_cluster'].value_counts()
pca_teams_samples['team_cluster'] = pca_teams_samples['team_cluster'].astype(str)

px.scatter(
  pca_teams_samples, 
  x='team_PC1', 
  y='team_PC2',
  color='team_cluster'
)

In [16]:
matches = pd.read_csv('../../assets/matches_atualizada.csv')
matches = matches.query('status == "complete"')[[
  'home_team_name',
  'away_team_name',
  'home_team_goal_count',
  'away_team_goal_count',
  'home_pre_match_xg',	
  'away_pre_match_xg',
  'home_ppg',
  'away_ppg',
  'average_goals_per_match_pre_match',
  'average_corners_per_match_pre_match',
  'btts_percentage_pre_match'
]]

matches = matches.merge(pca_teams_samples.drop(columns=['current_club']), left_on='home_team_name', right_on='common_name')
matches = matches.merge(pca_teams_samples.drop(columns=['current_club']), left_on='away_team_name', right_on='common_name', suffixes=('_home', '_away'))

In [17]:
matches_sample = matches.drop(columns=['common_name_home', 'common_name_away'])

# adiciona o vencedor da partida 1 = casa, 2 = fora, 0 = empate
matches_sample['winner'] = matches_sample.apply(lambda row: 1 if row['home_team_goal_count'] > row['away_team_goal_count'] else 2 if row['home_team_goal_count'] < row['away_team_goal_count'] else 0, axis=1)

# separa a mesma quantidade de casa, fora e empate de forma aleatória
home_wins = matches_sample.query('winner == 1')
draws = matches_sample.query('winner == 0')
away_wins = matches_sample.query('winner == 2')

cleaned_matches_sample = pd.concat([
  draws.sample(n=55, random_state=42), 
  away_wins.sample(n=55, random_state=42), 
  home_wins.sample(n=55, random_state=42)
])

matches_sample = pd.DataFrame(cleaned_matches_sample).reset_index(drop=True)

In [ ]:
matches_pca = PCA(n_components=4)

matches_features = matches_sample.drop(columns=[
  'home_team_name', 
  'away_team_name', 
  'home_team_goal_count', 
  'away_team_goal_count', 
  'winner'], axis=1)

matches_pca.fit(matches_features)
pca_matches_samples = matches_pca.transform(matches_features)
pca_matches_samples = pd.DataFrame(
  pca_matches_samples, 
  columns=[
    'matches_PC1', 
    'matches_PC2', 
    'matches_PC3', 
    'matches_PC4'])

pca_matches_samples['winner'] = matches_sample['winner']
pca_matches_samples['home_team_name'] = matches_sample['home_team_name']
pca_matches_samples['away_team_name'] = matches_sample['away_team_name']

# Plota gráfico Scree
fig = px.bar(x=range(1, len(matches_pca.explained_variance_ratio_)+1),
  y=matches_pca.explained_variance_ratio_,
  title='Gráfico Scree (Relação de Percentual de Variância e Componente Principal)',
  labels={'x':'Componente Principal', 'y':'Percentual de variância'})
fig.show()

# Treinamento do modelo

Após encontrar todos os clusters, reduzir dimencionalidade, realizar limpezas e normalizações e juntar tabelas, iniciamos o treinamento do modelo.

Para responder nossa pergunta em questão (Qual time ira ganhar?) vamos utilizar o Random Forest. Um modelo que gera varias arvores binárias e compara os resultados encontrados por elas.

> Random forest é um algoritmo de aprendizado de máquina amplamente utilizado, registrado por Leo Breiman e Adele Cutler, que combina o resultado de múltiplas árvores de decisão para chegar a um único resultado. Sua facilidade de uso e flexibilidade impulsionaram sua adoção, pois ele lida com problemas tanto de classificação quanto de regressão.

IBM. What is Random Forest? | IBM. Disponível em: <https://www.ibm.com/topics/random-forest#:~:text=Random%20forest%20is%20a%20commonly>.

In [ ]:
random_forest = RandomForestClassifier(n_estimators=100, random_state=42)

# separa 5 partidas para teste e remove do dataset de treino
manual_test = matches_sample.sample(n=5, random_state=42)
pca_matches_samples = pca_matches_samples.drop(manual_test.index)

X = pca_matches_samples.drop(columns=['winner', 'home_team_name', 'away_team_name'])
y = pca_matches_samples['winner']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_test.count()

random_forest.fit(X_train, y_train)

In [ ]:
y_pred = random_forest.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

matrix_conf = confusion_matrix(y_test, y_pred)

print(f'Acurácia: {(accuracy * 100):10.2f}%')
print(f'Matriz de confusão: \n{matrix_conf}')
plt.figure(figsize=(10, 7))
heat_map = sns.heatmap(matrix_conf, annot=True)

heat_map.set_xlabel('Previsto', fontsize=12)
heat_map.set_ylabel('Real', fontsize=12)

plt.show()

In [ ]:
X_test = X_test.merge(matches_sample.drop(columns=[
  'home_team_goal_count',
  'away_team_goal_count', 
  'cluster_0_home',
  'cluster_1_home', 
  'cluster_2_home', 
  'cluster_0_away',
  'cluster_1_away', 
  'cluster_2_away', 
]), left_index=True, right_index=True)

X_test['winner'] = y_test
X_test['predicted_winner'] = y_pred 

X_test.to_csv('./matches_test.csv', index=False)

X_test

In [ ]:
pca_matches_samples['winner'] = pca_matches_samples['winner'].astype(str)

px.scatter_3d(
  pca_matches_samples, 
  x='matches_PC1', 
  y='matches_PC2',
  z='matches_PC3',
  color='winner'
)

## Conclusão

Após realizar todos os treinamentos tratamentos e demais análises dos resultados do modelo, encontramos uma acurácia abaixo do esperado.
Como podemos ver na matriz de confusão que geramos acima, o modelo tem uma certa dificuldade ainda em diferencias vitórias da casa e vitórias do vistiante. Apesar de entender muito bem quando há empates. Ele indica muito menos empates do que as demais situações.

## Como aprimorar o modelo para encontrar melhores resultados?

Uma das soluçõe que podemos testar, está relacionada aos dados inputados no modelo. Apesar de terem uma correlação direta com a vitória, os dados não trazem uma boa diferenciação para o modelo e isso atrapalha o mesmo.

Podemos ainda realizar um tratamento no mesmo, gerando uma normalização para colunas que estão afetando negativamente o modelo e causando ruídos. A adição de dados que diferenciem as vitórias da casa e as vitórias do visitante também se mostram importante. Talvez podemos usar mais dados da casa e do visitante ao invés de dados gerais (não considerar "Gols Gerais" e sim "Gols marcados em casa").